#Open Library Analysis - Big Data Computing Project
####Graph's Connected Components vs. k-means Clusters

##Libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

##Data Aquisition

short dataset: 50000 entries

In [0]:
%sh wget -P /tmp https://raw.githubusercontent.com/attennig/BDC_datasets/main/books_short.csv

In [0]:
dbutils.fs.mv("file:/tmp/books_short.csv", "dbfs:/bdc-2020-21/datasets/books_short.csv")

long dataset: 22.589.356 entries 

remember to download it only if it is necessary

In [0]:
%sh wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qIhBSrpkDc-RCdbw7e1NVtNhOj_fNi5G' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1qIhBSrpkDc-RCdbw7e1NVtNhOj_fNi5G" -O /tmp/books_long.csv.bz2 && rm -rf /tmp/cookies.txt

In [0]:
dbutils.fs.ls("file:/tmp")

In [0]:
dbutils.fs.mv("file:/tmp/books_long.csv.bz2", "dbfs:/bdc-2020-21/datasets/books_long.csv.bz2")

In [0]:
dbutils.fs.ls("dbfs:/bdc-2020-21/datasets")

In [0]:
# Read dataset file into a Spark Dataframe
books_df = spark.read.load("dbfs:/bdc-2020-21/datasets/books_short.csv", 
                         format="csv", 
                         sep=";", 
                         inferSchema="true", 
                         header="true"
                         )

In [0]:
tot_entries = books_df.count();
print("The shape of the dataset is {:d} rows by {:d} columns".format(tot_entries, len(books_df.columns)))

The shape of the dataset is 50000 rows by 17 columns

##Data Cleaning

In [0]:
#eliminating entries with missing values or columns
columns_to_drop = []
for c in books_df.columns:
  if books_df.where(col(c).isNull()).count()/tot_entries > 0.7:
    #remove col
    columns_to_drop += [c]

In [0]:
columns_to_drop

In [0]:
books_df = books_df.drop(*columns_to_drop)

In [0]:
assert "title" in books_df.columns and "key" in books_df.columns
books_df = books_df.dropna(how="any", subset=["key", "title"])
books_df = books_df.dropDuplicates(['key'])
books_df = books_df.dropDuplicates(['title'])

In [0]:
assert "subjects" in books_df.columns and "authors" in books_df.columns
books_df = books_df.na.fill({'subjects': 'unknown', 'authors': 'unknown'})

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
# This will return a new DF with all the columns + id
books_df = books_df.withColumn("id", monotonically_increasing_id())

In [0]:
books_df.describe().show()

+-------+-----------+--------------------+-------------+--------------------+--------------------+
summary| key| title| authors| subjects| id|
+-------+-----------+--------------------+-------------+--------------------+--------------------+
 count| 49626| 49626| 49626| 49626| 49626|
 mean| null| 1501.6705882352942| null| null|2.063620297836797...|
 stddev| null| 737.069345249063| null| null|1.427163370577623...|
 min|OL10000079W| - die Hoffnung m...|['OL100070A']|["Accord Canada-O...| 0|
 max|OL23534178W|︠T︡siviliza︠t︡si︠...| unknown| unknown| 42949679532|
+-------+-----------+--------------------+-------------+--------------------+--------------------+

In [0]:
# if nltk not found
!python -m pip install nltk

!python -m pip install --upgrade pip

!python -m nltk.downloader all

In [0]:
# From Document_Clustering.ipynb
def clean_text(df, column_name):
    """ 
    This function takes the raw text data and applies a standard NLP preprocessing pipeline consisting of the following steps:
      - Text cleaning
      - Tokenization
      - Stopwords removal
      - Stemming (Snowball stemmer)

    parameter: dataframe
    returns: the input dataframe along with the `cleaned_content` column as the results of the NLP preprocessing pipeline
    """
    from pyspark.sql.functions import udf, col, lower, trim, regexp_replace
    from pyspark.ml.feature import Tokenizer, StopWordsRemover
    from nltk.stem.snowball import SnowballStemmer # BE SURE NLTK IS INSTALLED ON THE CLUSTER USING THE "LIBRARIES" TAB IN THE MENU

    
    # Text preprocessing pipeline
    # 1. Text cleaning
    # 1.a Case normalization
    lower_case_df = df.select(["id",lower(col(column_name)).alias(column_name)])
    # 1.b Trimming
    trimmed_df = lower_case_df.select(["id",trim(col(column_name)).alias(column_name)])
    # 1.c Filter out punctuation symbols
    no_punct_df = trimmed_df.select(["id",(regexp_replace(col(column_name), "[^a-zA-Z\\s]", "")).alias(column_name)])
    # 1.d Filter out any internal extra whitespace
    cleaned_df = no_punct_df.select(["id",trim(regexp_replace(col(column_name), " +", " ")).alias(column_name)])
    # 2. Tokenization (split text into tokens)
    tokenizer = Tokenizer(inputCol=column_name, outputCol="tokens")
    tokens_df = tokenizer.transform(cleaned_df).cache()
    # 3. Stopwords removal
    stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="terms")
    terms_df = stopwords_remover.transform(tokens_df).cache()
    # 4. Stemming (Snowball stemmer)
    stemmer = SnowballStemmer(language="english")
    stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
    terms_stemmed_df = terms_df.withColumn("terms_stemmed", stemmer_udf("terms")).cache()

    return terms_stemmed_df

In [0]:
clean_title_df = clean_text(books_df, "title")

In [0]:
clean_subjects_df = clean_text(books_df, "subjects")

##Feature Engineering

In this section we will use nlp techniques to get numerical vectors that represent text-based features

In [0]:
# final df
# |id|w2vec(clean_title)|w2vec(clean_subject)|authors|

In [0]:
RANDOM_SEED = 42 # used below to run the actual K-means clustering
EMBEDDING_SIZE = 150 # size of embedding Word2Vec vectors

In [0]:
#Word2Vec from Document_Clustering.ipynb 
def extract_w2v_features(df, column_name, out_col_name):
  from pyspark.ml.feature import Word2Vec
  
  word2vec = Word2Vec(vectorSize=EMBEDDING_SIZE, minCount=5, inputCol=column_name, outputCol=out_col_name, seed=RANDOM_SEED)
  model = word2vec.fit(df)
  features = model.transform(df).cache()
  
  return model, features

In [0]:
model, w2v_title_features = extract_w2v_features(clean_title_df, "terms_stemmed", "title_vec")

In [0]:
model, w2v_subjects_features = extract_w2v_features(clean_subjects_df, "terms_stemmed", "subjects_vec")

In [0]:
# Gli ID sono differenti, questo è un problema!!!!!!
books_df.show()
w2v_title_features.show()

+-----------+--------------------+--------------+--------------------+---+
 key| title| authors| subjects| id|
+-----------+--------------------+--------------+--------------------+---+
OL17045571W| !Claveles rojos!|['OL7225474A']| unknown| 0|
OL16850144W| !Mi Granada!|['OL7182471A']| unknown| 1|
OL16899391W| !Pícara lengua!|['OL5316652A']| unknown| 2|
OL19558644W|" Teletubbies" An...| unknown| unknown| 3|
OL21899419W|"Beijing wen xue"...|['OL4176669A']|['Social conditio...| 4|
OL13334368W|"Belye pi︠a︡tna" ...|['OL6230186A']|['Spies', 'Histor...| 5|
OL10468844W|"Buy your own che...|['OL4341883A']| unknown| 6|
OL16827569W|"In trouble", chi...| unknown|['Community and s...| 7|
OL11466013W|"Intifadat Al-Aḳt...|['OL4825996A']|['Palestinian Ara...| 8|
OL15930512W|"Minnesota Fats" ...|['OL6939949A']| ['Pool (Game)']| 9|
OL10551958W|'A case study of ...|['OL4381027A']| unknown| 10|
OL19097182W|'Cashville'' - Di...|['OL7471742A']| unknown| 11|
OL10638796W|'On the brink of ...|['OL4430620A']| unknown| 12|
 OL1339666W|(The roaring girl...| ['OL136454A']| unknown| 13|
OL14952864W|- Maggie Lena Wal...| ['OL529549A']| unknown| 14|
OL17616956W| ... Adresse|['OL4548854A']|['Foreign relatio...| 15|
 OL1237509W|... Museum catalo...| ['OL124914A']| unknown| 16|
OL23340247W|1 thotsawat Saphā...|['OL8853371A']|['Industries', 'F...| 17|
 OL1749094W|1+1+1 Une Révolution| ['OL202533A']|['Politics and go...| 18|
OL17021353W|1,001 ways to get...| ['OL26034A']|['Success in busi...| 19|
+-----------+--------------------+--------------+--------------------+---+
only showing top 20 rows

+---+--------------------+--------------------+--------------------+--------------------+--------------------+
 id| title| tokens| terms| terms_stemmed| title_vec|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+
 0|the design and ev...|[the, design, and...|[design, evaluati...|[design, evalu, v...|[-0.0129747303823...|
 1|central service t...|[central, service...|[central, service...|[central, servic,...|[6.07432360993698...|
 2|mmpi in testing m...|[mmpi, in, testin...|[mmpi, testing, m...|[mmpi, test, medi...|[-0.0259415877517...|
 3|recent developmen...|[recent, developm...|[recent, developm...|[recent, develop,...|[-0.0132321310666...|
 4|quantum groups an...|[quantum, groups,...|[quantum, groups,...|[quantum, group, ...|[-0.0199462290620...|
 5|cognitive assessm...|[cognitive, asses...|[cognitive, asses...|[cognit, assess, ...|[-0.0098034138791...|
 6|marginal space le...|[marginal, space,...|[marginal, space,...|[margin, space, l...|[-0.0190285191032...|
 7| letras francesas| [letras, francesas]| [letras, francesas]| [letra, francesa]|[0.00349456048570...|
 8| pamietnik zza grobu|[pamietnik, zza, ...|[pamietnik, zza, ...|[pamietnik, zza, ...|[0.0,0.0,0.0,0.0,...|
 9|zum gedenken an d...|[zum, gedenken, a...|[zum, gedenken, d...|[zum, gedenken, d...|[0.02470755204558...|
 10|ley de sociedades...|[ley, de, socieda...|[ley, de, socieda...|[ley, de, socieda...|[-0.0625290413387...|
 11| | []| []| []|[0.00302761234343...|
 12|medals and decora...|[medals, and, dec...|[medals, decorati...|[medal, decor, br...|[-0.0084072400738...|
 13| fedeli damore| [fedeli, damore]| [fedeli, damore]| [fede, damor]|[0.0,0.0,0.0,0.0,...|
 14| riat alunth| [riat, alunth]| [riat, alunth]| [riat, alunth]|[0.0,0.0,0.0,0.0,...|
 15|l amour de cupido...|[l, amour, de, cu...|[l, amour, de, cu...|[l, amour, de, cu...|[-0.0514658702470...|
 16|the burials of co...|[the, burials, of...|[burials, congreg...|[burial, congreg,...|[-0.0016119074910...|
 17| great eagles| [great, eagles]| [great, eagles]| [great, eagl]|[-0.0154981858795...|
 18|research on acupu...|[research, on, ac...|[research, acupun...|[research, acupun...|[-0.0016889652237...|
 19|fizikokhimicheski...|[fizikokhimichesk...|[fizikokhimichesk...|[fizikokhimichesk...|[-2.2222189831414...|
+---+--------------------+--------------------+--------------------+------------------

In [0]:
# Final dataframe
engineered_books_df = books_df.select(["id", "authors"])
# add "title_vec" and "subjects_vec"

In [0]:
#engineered_books_df = engineered_books_df.withColumn(w2v_title_features["title_vec"])

In [0]:
#engineered_books_df = engineered_books_df.withColumn("subjects_vec", w2v_subjects_features["features"])

##Graph

In [0]:
# guardare notebook pagerank
# creare df con colonna id per i nodi
# creare df con colonne src e dst per gli archi
# def sim(u,v)
# gli archi vanno agginti bidirezionali per le entries che rispettalo la condizione sim(u,v) > eps
# crea grafo
# ottieni CC del grafo


##Clustering

In [0]:
# from Document_Clustering.ipynb  e l'altro del clustering
# k = nCC , kmeans

##Evaluation